In [ ]:
#@title Colab setup
import os, sys, subprocess
if "google.colab" in sys.modules:
  cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic intersect"
  process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
# ------

# Librerias que utilizaremos para resolver ODEs
import scipy.integrate
import biocircuits
import numba

# Librerias para analisis matematico
import numpy as np
from intersect import intersection
from skimage import measure
import sympy as sym
import pylab as pl

# Librerias para visualizar los resultados
import bokeh.io
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.models import Range1d
from bokeh.io import export_svgs

bokeh.io.output_notebook()

In [ ]:
# Autores: Frank Britto Bisso, revisado por PhD. Christian Cuba Samaniego
# Fecha: 15 de Julio, 2024
# 2024 Workshop en Modelamiento de Sistemas Biológicos

# Día 1. Introducción al modelamiento de sistemas biológicos



## Módulo 1. Intuición detrás del modelamiento de circuitos genéticos

Estudiamos la producción de una proteína $Y$ a partir de la expresión de un gen, representada en la siguiente imagen.

<p align="center">
  <img src="https://biocircuits.github.io/_images/simplest_protein.png" alt="Protein production" width="300">
</p>

Este fenómeno puede ser capturado mediante las siguientes reacciones químicas:

\begin{eqnarray}
  \emptyset \xrightarrow{\alpha} Y \\
  Y \xrightarrow{\delta} \emptyset
\end{eqnarray}

Utilizamos la ley de acción de masas para modelar las reacciones químicas mediante una ecuación diferencial ordinaria (EDO) de primer orden:

\begin{eqnarray}
  \frac{dy}{dt}= \text{producccion} - \text{degradacion} = \alpha - \delta y
\end{eqnarray}

donde $\alpha$ es la tasa de producción máxima de la proteína $Y$, y $\delta$ es la tasa de degradación de dicha proteína, que considera también la dilución de la concentración por la división celular.

In [ ]:
# Definimos una función que captura las EDOs

def protein_expression(X, t, a, d):
  """
  Las expresiones al lado derecho muestran la producción y degradación.
  El output de la función será dy / dt
  """

  y = X

  return (a - d*y)

Nos interesa simular cómo varía la concentración de la proteína $y$ en el tiempo.

In [ ]:
# Definimos el entorno de visualización en bokeh
p = bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 20),
                          y_range = Range1d(0, 2.5),
                          x_axis_label = 'Tiempo [h]',
                          y_axis_label = 'y [uM]')

# Definimos las condiciones iniciales
x01 = np.array([0.])

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
args = (a, d)

# Definimos el tiempo de simulación
t = np.linspace(0,20,200)

# Resolvemos
x1 = scipy.integrate.odeint(protein_expression, x01, t, args = args)
x1 = x1.transpose()

# Ploteamos
p.line(t, x1[0], line_width = 3, color = 'black')
p.title.text = 'Expresión de la proteína Y'

# Visualizamos
bokeh.io.show(p)

In [ ]:
#@title Simulaciones estocásticas de la producción de proteínas

# Stoichiometry matrix
simple_update = np.array(
    [
        [ 1], # 0 -> Y
        [-1], # Y -> 0
    ]
)

# Propensities matrix
def simple_propensity(propensities, population, t, a, d, Omega):
  y = population[0]

  propensities[0] = a * Omega
  propensities[1] = d * y

# Definimos el entorno de visualización en bokeh
p = bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 20),
                          x_axis_label = 'Tiempo [h]',
                          y_axis_label = 'y [# moléculas]')

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]

# Definimos los parámetros del algoritmo de Gillespie
Omega = 100 # 600
time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(1, dtype=int) # condiciones iniciales
args1 = (a, d, Omega)

# Resolvemos mediante algoritmo de Gillespie
samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=10, args=args1, n_threads=2,progress_bar=False)

# Ploteamos la dinámica de Y
for i in range(20):
  p.line(time_points, samples[i,:,0], line_width = 1, color = 'black', alpha = 0.2, line_join = 'bevel')
p.line(time_points, np.mean(samples[:,:,0], axis = 0), line_width = 3, color = 'orange')
p.title.text = 'Expresión de la proteína Y'

# Visualizamos
bokeh.io.show(p)

En otro ejemplo, intentemos modelar la transcripción y la traducción como pasos separados.

\begin{eqnarray}
  \emptyset \xrightarrow{\alpha} \text{mRNA} && \\
  \text{mRNA} \xrightarrow{\delta} \emptyset \\
  \text{mRNA} \xrightarrow{\beta} Y \\
  Y \xrightarrow{\phi} \emptyset
\end{eqnarray}

Aplicando la ley de acción de masas,

\begin{eqnarray}
  \frac{dm_y}{dt} &=& \alpha - \delta m_y \\
  \frac{dy}{dt} &=& \beta m_y - \phi y
\end{eqnarray}

In [ ]:
#@title Modelando transcripción y traducción (TX-TL)

def txtl(X, t, a, d, b, p):
  """
  Las expresiones al lado derecho muestran la producción y degradación.
  El output de la función será dy / dt
  """

  m, y = X

  return (
      [
          a - d*m,
          b*m - p*y
      ]
  )

# Definimos el entorno de visualización en bokeh
plots = []
for i in range(2):
  plots.append(bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 100),
                          #y_range = Range1d(0, 3.0),
                          x_axis_label = 'Tiempo [h]',
  ))

# Definimos las condiciones iniciales
x01 = np.array([0., 0.])

# Definimos los parámetros del modelo
a = 1 # tasa de producción de mRNA [uM/h]
d = 0.1 # tasa de degradación de mRNA [1/h]
b = 2 # tasa de producción de proteína y [uM/h]
p = 0.05 # tasa de degradación de proteína y [1/h]
args = (a, d, b, p)

# Definimos el tiempo de simulación
t = np.linspace(0,100,200)

# Resolvemos
x1 = scipy.integrate.odeint(txtl, x01, t, args = args)
x1 = x1.transpose()

# Ploteamos
plots[0].line(t, x1[0,:], line_width = 3, color = 'black')
plots[0].title.text = 'Expresión de mRNA'
plots[0].yaxis.axis_label = 'mRNA [um/h]'

plots[1].line(t, x1[1,:], line_width = 3, color = 'orange')
plots[1].title.text = 'Expresión de y'
plots[1].yaxis.axis_label = 'y [um/h]'

# Visualizamos
bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
#@title Simulaciones estocásticas del sistema TX-TL

# Stoichiometry matrix
simple_update = np.array(
    [
        [ 1, 0], # 0 -> mRNA
        [-1, 0], # mRNA -> 0
        [ 0, 1], # 0 -> Y
        [ 0,-1], # Y -> 0
    ]
)

# Propensities matrix
def simple_propensity(propensities, population, t, a, d, Omega):
  m, y = population

  propensities[0] = a * Omega
  propensities[1] = d * m
  propensities[2] = a * m
  propensities[3] = d * y

# Definimos el entorno de visualización en bokeh
plots = []
for i in range(2):
  plots.append(bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 100),
                          x_axis_label = 'Tiempo [h]',
                          y_axis_label = 'y [# moléculas]'))

# Definimos los parámetros del modelo
a = 1 # tasa de producción de mRNA [uM/h]
d = 0.1 # tasa de degradación de mRNA [1/h]
b = 2 # tasa de producción de proteína y [uM/h]
p = 0.05 # tasa de degradación de proteína y [1/h]

# Definimos los parámetros del algoritmo de Gillespie
Omega = 100 # 600
time_points = np.linspace(0, 100, 201)
population_0 = np.zeros(2, dtype=int) # condiciones iniciales
args1 = (a, d, Omega)

# Resolvemos mediante algoritmo de Gillespie
samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=10, args=args1, n_threads=2,progress_bar=True)

# Ploteamos la dinámica de mRNA
for i in range(20):
  plots[0].line(time_points, samples[i,:,0], line_width = 1, color = 'black', alpha = 0.2, line_join = 'bevel')

plots[0].line(time_points, np.mean(samples[:,:,0], axis = 0), line_width = 3, color = 'orange', alpha = 1, line_join = 'bevel')
plots[0].title.text = 'Dinámica de mRNA'

# Ploteamos la dinámica de proteína y (visualización alternativa)
low = np.percentile(samples[:, :, 1], 2.5, axis=0)
high = np.percentile(samples[:, :, 1], 97.5, axis=0)

plots[1].patch(
      np.concatenate((time_points, time_points[::-1])),
      np.concatenate((low, high[::-1])),
      alpha=0.3,
      line_alpha=0,
      color = 'grey'
  )

plots[1].line(time_points, np.mean(samples[:,:,1], axis = 0), line_width = 3, color = 'orange', alpha = 1, line_join = 'bevel')
plots[1].title.text = 'Dinámica de proteína Y'

# Visualizamos
bokeh.io.show(bokeh.layouts.row(plots))

100%|██████████| 10/10 [01:02<00:00,  6.30s/it]


## Módulo 2. Regulación de genes: ultrasensibilidad y funciones de Hill

De acuerdo al tipo de interacción entre el factor de transcripción y el DNA binding site, podemos tener activadores y represores, los cuáles son modelados mediante funcions de Hill, tal que

* Activación por factor de transcripción: $\alpha \frac{x^m}{K^m + x^m}$, donde $\alpha$ es la máxima tasa de producción, $K$ es la constante de disociación, $x$ es la concentración del factor de transcripción en cuestión, y $m$ es el coeficiente de hill, también llamado cooperatividad
* Represión por factor de transcripción: $\alpha \frac{K^m}{K^m + x^m}$, considerando los mismos parámetros

**Nota:** al utilizar estas expresiones, estamos realizando una operación llamada separación de escala de tiempo (*time scale separation*), donde consideramos que la asociación y disociación de los factores de transcripción a su sitio de unión en el ADN son mucho más rápidos que la tasa a la que varía la producción de mRNA y la proteína $y$.

Comenzaremos modelando un gen que produce un factor de transcripción $X$ que reprime su propia expresión (*self-inhibiting gene*). Considerando las reacciones químicas:

\begin{eqnarray}
  \emptyset \xrightarrow{h(x)} X \\
  X \xrightarrow{\delta} \emptyset
\end{eqnarray}

donde $h(x)$ representa la regulación mediada por el factor de transcripción. Utilizando la ley de acción de masas,

\begin{eqnarray}
  \frac{dx}{dt} = \alpha \frac{K^m}{K^m + x^m} - \delta x
\end{eqnarray}

In [ ]:
# Definimos una función que captura las EDOs

def self_inhibiting(X, t, a, d, K, m):
  """
  Las expresiones al lado derecho muestran la producción y degradación.
  El output de la función será dy / dt
  """

  x = X

  return (a * K**m / (K**m + x**m) - d*x)

Adaptamos el procedimiento detallado anteriormente.

In [ ]:
# Definimos el entorno de visualización en bokeh
plots = []
for i in range(2):
  plots.append(bokeh.plotting.figure(width = 325, height = 275,))

# Agregamos una paleta de colores
colors = bokeh.palettes.Greys8[::-1]

# Definimos las condiciones iniciales
x01 = np.array([0.])

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 0.1 # constante de disociación [uM]
m = 3 # cooperatividad

# Definimos el tiempo de simulación
t = np.linspace(0,20,200)

# Vamos a variar la cooperatividad para estudiar su efecto en la respuesta
vec_m = np.array([1,2,3,5,10])

# Resolvemos
for i, mi in enumerate(vec_m):
  args = (a, d, K, mi)
  x1 = scipy.integrate.odeint(self_inhibiting, x01, t, args = args)
  x1 = x1.transpose()

  # Ploteamos
  plots[0].line(t, x1[0], line_width = 3, color = colors[i+2], legend_label = f'm={mi}')

# Algunos detalles extra en el plot
plots[0].title.text = 'Dinámica de un represor'
plots[0].xaxis.axis_label = 'Tiempo [h]'
plots[0].yaxis.axis_label = 'x [uM]'

# Curva dosis-respuesta
xi = np.linspace(0,1,200)
for i, mi in enumerate(vec_m):
  repressor = a * K**mi / (K**mi + xi**mi)
  plots[1].line(xi/K, repressor, line_width = 3, color = colors[i+2])

# Algunos ajustes para ver mejor la gráfica
plots[1].x_range = Range1d(0, 6)
plots[1].title.text = 'Curva dosis-respuesta'
plots[1].xaxis.axis_label = 'x/K'
plots[1].yaxis.axis_label = 'h(x)'

# Visualizamos
bokeh.io.show(bokeh.layouts.row(plots))

Como siguiente ejemplo, podemos modelar un gen que expresa un factor de transcripción que induce su propia expresión.

In [ ]:
#@title Self-activating gene

# Definimos una función que captura las EDOs

def self_inducing(X, t, a, d, K, m):
  """
  Las expresiones al lado derecho muestran la producción y degradación.
  El output de la función será dy / dt
  """

  x = X

  return (a * x**m / (K**m + x**m) - d*x)

# Definimos el entorno de visualización en bokeh
plots = []
for i in range(2):
  plots.append(bokeh.plotting.figure(width = 325, height = 275,))

# Agregamos una paleta de colores
colors = bokeh.palettes.Oranges8[::-1]

# Definimos las condiciones iniciales
x01 = np.array([0.1]) # Pensemos: ¿por qué no podemos setear las condiciones iniciales en 0?

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 0.1 # constante de disociación [uM]
m = 3 # cooperatividad

# Definimos el tiempo de simulación
t = np.linspace(0,20,200)

# Vamos a variar la cooperatividad para estudiar su efecto en la respuesta
vec_m = np.array([1,2,3,5,10])

# Resolvemos
for i, mi in enumerate(vec_m):
  args = (a, d, K, mi)
  x1 = scipy.integrate.odeint(self_inducing, x01, t, args = args)
  x1 = x1.transpose()

  # Ploteamos
  plots[0].line(t, x1[0], line_width = 3, color = colors[i+2], legend_label = f'm={mi}')

# Algunos detalles extra en el plot
plots[0].title.text = 'Dinámica de un activador'
plots[0].xaxis.axis_label = 'Tiempo [h]'
plots[0].yaxis.axis_label = 'x [uM]'

# Curva dosis-respuesta
xi = np.linspace(0,1,200)
for i, mi in enumerate(vec_m):
  inducer = a * xi**mi / (K**mi + xi**mi)
  plots[1].line(xi/K, inducer, line_width = 3, color = colors[i+2])

# Algunos ajustes para ver mejor la gráfica
plots[1].x_range = Range1d(0, 6)
plots[1].title.text = 'Curva dosis-respuesta'
plots[1].xaxis.axis_label = 'x/K'
plots[1].yaxis.axis_label = 'h(x)'

# Visualizamos
bokeh.io.show(bokeh.layouts.row(plots))

## Módulo 3. Sistemas biestables: toggle switch

El *toggle switch*, diseñado por Gardner y Collins en el 2000, plantea dos genes que expresan factores de transcripción que se reprimen mutuamente. Podemos modelar este sistema mediante las siguientes ecuaciones químicas:

\begin{eqnarray}
  \emptyset \xrightarrow{h(y_2)} Y_1 && \\
  Y_1 \xrightarrow{\delta} \emptyset \\
  \emptyset \xrightarrow{h(y_1)} Y_2 \\
  Y_2 \xrightarrow{\phi} \emptyset
\end{eqnarray}

Utilizando la ley de acción de masas, obtenemos las siguientes EDOs:

\begin{eqnarray}
  \frac{dy_1}{dt} &=& \alpha \frac{K^m}{K^m + y_2^m} - \delta y_1 \\
  \frac{dy_2}{dt} &=& \alpha \frac{K^m}{K^m + y_1^m} - \delta y_2
\end{eqnarray}

In [ ]:
# Definimos una función que captura las EDOs

def toggle_switch(X, t, a, d, K, m):
  """
  Las expresiones al lado derecho muestran la producción y degradación.
  El output de la función será dy / dt
  """

  y1, y2 = X

  return (
      [
          a * K**m / (K**m + y2**m) - d*y1, # d y1 / dt
          a * K**m / (K**m + y1**m) - d*y2, # d y2 / dt
      ]
  )

Adaptamos el procedimiento detallado anteriormente.

In [ ]:
# Definimos el entorno de visualización en bokeh
plots = []
for i in range(2):
  plots.append(bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 20),
                          x_axis_label = 'Tiempo [h]',
                          y_axis_label = 'y [uM]'))

# Definimos las condiciones iniciales
x01 = np.array([0., 0.1])

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 1 # constante de disociación [uM]
m = 3 # cooperatividad
args = (a, d, K, m)

# Definimos el tiempo de simulación
t = np.linspace(0,20,200)

# Resolvemos
x1 = scipy.integrate.odeint(toggle_switch, x01, t, args = args)
x1 = x1.transpose()

# Ploteamos
plots[0].line(t, x1[0,:], line_width = 3, color = 'black', legend_label = 'y1')
plots[0].line(t, x1[1,:], line_width = 3, color = 'orange', legend_label = 'y2')
plots[0].title.text = 'Dinámica del toggle switch'

# Estudiemos con más detalles este sistema para diferentes condiciones iniciales
vecA = np.linspace(1,3,4)
vecB = np.ones(4)*2.9
vecC = np.copy(vecA)
vecC[::1] += 0.1
vec_y1 = np.concatenate([vecB, vecA])
vec_y2 = np.concatenate([vecA, vecC])

# Resolvemos las EDOs
for ii in range(len(vec_y1)):

  # Aquí definimos las diferentes condiciones iniciales
  x01 = np.array([vec_y1[ii] , vec_y2[ii], ])

  # Solving the ODEs
  x1 = scipy.integrate.odeint(toggle_switch, x01, t, args=args)
  x1 = x1.transpose()

  # Ploting
  plots[1].line(t,x1[0,:],line_width=2, color="black", )
  plots[1].line(t,x1[1,:],line_width=2, color="orange",)

plots[1].title.text = 'El toggle switch tiene dos estados estables'

# Visualizamos
bokeh.io.show(bokeh.layouts.row(plots))

La dinámica del toggle switch que simulamos nos parece indicar que, en estado estacionario (i.e, $\text{tiempo} \to \infty$) el toggle switch puede adoptar uno de dos estados. Un gráfico que nos ayuda a analizar este comportamiento es el *phase plane*, el cuál nos permite comparar la dinámica de una especie con respecto a otra(s).

Obtenemos este gráfico al resolver $dy_1/dt = 0$ y $dy_2/dt = 0$. Cada una de las soluciones representa la llamada ecuación del *nullclines* de cada especie. Si evaluamos la intersección de estos *nullclines* en el plano, encontraremos los puntos de equilibrio del sistema.

\begin{eqnarray}
  \bar y_1 = \frac{\alpha}{\delta} \frac{K^m}{K^m + \bar y_2^m} \\
  \bar y_2 = \frac{\alpha}{\delta} \frac{K^m}{K^m + \bar y_1^m}
\end{eqnarray}

In [ ]:
# Definimos las ecuaciones de nullclines de cada especie

def null_y1(y2_2, a, d, K, m):

  y1_2 = (a/d) * K**m / (K**m + y2_2**m)

  return y1_2

def null_y2(y1_1, a, d, K, m):

  y2_1 = (a/d) * K**m / (K**m + y1_1**m)

  return y2_1

In [ ]:
# Definimos el entorno de visualización en bokeh
plots = []
for i in range(2):
  plots.append(bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 3.1),
                          y_range = Range1d(0, 3.1),
                          x_axis_label = 'y1',
                          y_axis_label = 'y2'))

# Definimos las variables de estados
y1_1 = np.linspace(0,3,300)
y2_2 = np.linspace(0,3,300)

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 1 # constante de disociación [uM]
m = 3 # cooperatividad

# Calculamos las expresiones del nullclines
y1_2 = null_y1(y2_2, a, d, K, m)
y2_1 = null_y2(y1_1, a, d, K, m)

# Podemos hallar la intersección mediante la función intersect
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

# Ploteamos
for i in range(2):
  plots[i].line(y1_1, y2_1, line_width = 3, color = 'black', alpha = 0.75)
  plots[i].line(y1_2, y2_2, line_width = 3, color = 'black', alpha = 0.75)
  plots[i].circle(y1_i, y2_i, line_width = 2, size = 8, color = 'black', fill_color = 'white')
plots[0].title.text = 'Phase plane del toggle switch'

# Simulando diferentes condiciones iniciales
vec = np.linspace(0,3,21)
tmp1 = np.stack((vec*0 + 0.01, vec), axis=1)
tmp2 = np.stack((vec*0 + vec[-1] - 0.01, vec), axis=1)
tmpA = np.concatenate((tmp1, tmp2), axis=0)

tmp1 = np.stack((vec, vec*0 + 0.01), axis=1)
tmp2 = np.stack((vec,vec*0 + vec[-1] - 0.01), axis=1)
tmpB = np.concatenate((tmp1, tmp2), axis=0)

tmp = np.concatenate((tmpA, tmpB), axis=0)

# Podemos observar hacia donde convergen las soluciones
args = (a, d, K, m)
for x0i in tmp:
  x0 = np.array([ x0i[0], x0i[1],]) # condiciones iniciales
  sol = scipy.integrate.odeint(toggle_switch, x0, t, args=args)
  sol = sol.transpose()
  ratio = sol[0,-1]/sol[1,-1]
  if ratio > 1:
      plots[1].line(sol[0,:],sol[1,:],line_width=1, color="grey",) #
  elif ratio < 1:
      plots[1].line(sol[0,:],sol[1,:],line_width=1, color="orange",) #
  else:
      plots[1].line(sol[0,:],sol[1,:],line_width=1, color="black",) #
plots[1].title.text = 'Toggle switch converge a dos estados estables'

# Visualizamos
bokeh.io.show(bokeh.layouts.row(plots))

## Módulo 4. ¿Cómo implementar las simulaciones estocásticas?

Si bien las simulaciones determinísticas nos permiten obtener una mejor idea sobre la cinética de nuestros circuitos genéticos y determinar principios de diseño, en la realidad, la expresión de genes es un proceso estocástico. Para realizar simulaciones que consideren esta naturaleza aleatoria y ruidosa, con cierto grado de incertidumbre, utilizaremos el algoritmo de Gillespie.


In [ ]:
# Stoichiometry matrix

simple_update = np.array(
    [
        [ 1, 0], # 0 -> Y1
        [-1, 0], # Y1 -> 0
        [ 0, 1], # 0 -> Y2
        [ 0,-1], # Y2 -> 0
    ]
)

In [ ]:
# Stoichiometry matrix

simple_update = np.array(
    [
        [ 1, 0], # 0 -> Y1
        [-1, 0], # Y1 -> 0
        [ 0, 1], # 0 -> Y2
        [ 0,-1], # Y2 -> 0
    ]
)

# Propensities matrix

def simple_propensity(propensities, population, t, a, d, K, m, Omega):
  y1, y2 = population

  propensities[0] = a * Omega * (K*Omega)**m / ((K*Omega)**m + y2**m)
  propensities[1] = d * y1
  propensities[2] = a * Omega * (K*Omega)**m / ((K*Omega)**m + y1**m)
  propensities[3] = d * y2


Utilizaremos el paquete ```biocircuits``` para implementar el algoritmo de Gillespie.



In [ ]:
# Definimos el entorno de visualización en bokeh
plots = []
for i in range(2):
  plots.append(bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 20),
                          x_axis_label = 'Tiempo [h]',
                          y_axis_label = 'y [# moléculas]'))

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 1 # constante de disociación [uM]
m = 3 # cooperatividad

# Definimos los parámetros del algoritmo de Gillespie
Omega = 100 # 600
time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(2, dtype=int) # condiciones iniciales
args1 = (a, d, K, m, Omega)

# Resolvemos mediante algoritmo de Gillespie
samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=10, args=args1, n_threads=2,progress_bar=True)

# Ploteamos la dinámica de Y1
for i in range(20):
  plots[0].line(time_points, samples[i,:,0], line_width = 1, color = 'black', alpha = 0.2, line_join = 'bevel')
  plots[1].line(time_points, samples[i,:,1], line_width = 1, color = 'black', alpha = 0.2, line_join = 'bevel')

plots[0].title.text = 'Dinámica de Y1'
plots[1].title.text = 'Dinámica de Y2'

# Visualizamos
bokeh.io.show(bokeh.layouts.row(plots))

100%|██████████| 10/10 [00:05<00:00,  1.68it/s]


Adicionalmente, evaluemos los resultados en el *phase plane*.

In [ ]:
#@title Distribución de probabilidades

# Definimos el entorno de visualización en bokeh
p1 = bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 3.1),
                          y_range = Range1d(0, 3.1),
                          x_axis_label = 'y1',
                          y_axis_label = 'y2')

# Definimos las variables de estados
y1_1 = np.linspace(0,3,300)
y2_2 = np.linspace(0,3,300)

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 1 # constante de disociación [uM]
m = 3 # cooperatividad

# Calculamos las expresiones del nullclines
y1_2 = null_y1(y2_2, a, d, K, m)
y2_1 = null_y2(y1_1, a, d, K, m)

# Podemos hallar la intersección mediante la función intersect
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

# Ploteamos
p1.line(y1_1, y2_1, line_width = 3, color = 'black', alpha = 0.75)
p1.line(y1_2, y2_2, line_width = 3, color = 'black', alpha = 0.75)
p1.circle(y1_i, y2_i, line_width = 2, size = 8, color = 'black', fill_color = 'white')
p1.title.text = 'Trayectorias'

# Utilizando las simulaciones estocásticas anteriores
for i in range(10):
    sol = samples[i,:,:]/Omega
    ratio = sol[-1,0]/sol[-1,1]
    if ratio > 1:
        p1.line(sol[:,0],sol[:,1],line_width=2, color="grey",alpha= 0.4, line_join="bevel") #
    elif ratio < 1:
        p1.line(sol[:,0],sol[:,1],line_width=2, color="orange",alpha= 0.4, line_join="bevel") #
    else:
        p1.line(sol[:,0],sol[:,1],line_width=2, color="black",alpha= 0.4, line_join="bevel")

# Definimos uno nuevo entorno de visualización en bokeh
p2 = bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(-300, 300),
                          y_range = Range1d(0, 0.032),
                          x_axis_label = 'y1-y2',
                          y_axis_label = 'p')

# Distribución de probabilidades
bin0 = np.arange(-3*Omega,3*Omega+6,8) #8
hist, bin_edges = np.histogram(samples[:,-1,0] - samples[:,-1,1], bin0, density=True)
p2.quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],fill_color="navy", line_color="navy", fill_alpha=0.2,line_alpha=0.)
p2.step(bin_edges[:-1],hist, line_width=1, mode="after", color = "navy",)
p2.title.text = 'Distribución de probabilidades'

# Visualizamos
bokeh.io.show(bokeh.layouts.row([p1, p2]))

## Tarea. Expresión de un represor considerando TX-TL

Simule determinística y estocásticamente el gene que produce un represor que inhibe su propia expresión (*self-inhibiting gene*), considerando los pasos de transcripción y traducción.

En este modelo, consideramos las siguientes ecuaciones químicas. Por notación, $p^*$ es el promotor libre (y por lo tanto, se puede realizar la transcripción), $p$ es el promotor unido al factor de transcripción $y$, formando un complejo, $m$ es el mRNA, e $y$ es el represor.

\begin{eqnarray}
  p^* \xrightarrow{\alpha} m \\
  m \xrightarrow{\delta} \emptyset \\
  p^* + y \xrightarrow{k_{on}} p \\
  p \xrightarrow{k_{off}} p^* + y \\
  m \xrightarrow{\beta} y \\
  y \xrightarrow{\phi} \emptyset
\end{eqnarray}

Utilizando la ley de acción de masas:

\begin{eqnarray}
  \frac{dm}{dt} &=& \alpha p^* - \delta m \\
  \frac{dy}{dt} &=& \beta m + k_{off}p - k_{on}yp^* - \phi y \\
  \frac{dp^*}{dt} &=& k_{off}p - k_{on}yp^* \\
  \frac{dp}{dt} &=& k_{on}yp^* - k_{off}p
\end{eqnarray}

In [ ]:
#@title Simulaciones determinísticas

# Definimos una función que captura las EDOs
def repressor_txtl(X, t, a, d, b, p, K, m, kon, koff):
  """
  Las expresiones al lado derecho muestran la producción y degradación.
  El output de la función será dy / dt
  """

  m, y, p_free, p = X

  return (
      [
          a*p_free - d*m, # d mRNA / dt
          b*m + koff*p - kon*y*p_free - p*y, # d y / dt
          koff*p - kon*y*p_free, # d p^* / dt
          kon*y*p_free - koff*p, # d p / dt
      ]
  )

# Definimos el entorno de visualización en bokeh
plots = []
for i in range(2):
  plots.append(bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 40),
                          #y_range = Range1d(0, 3.0),
                          x_axis_label = 'Tiempo [h]',
  ))

# Definimos las condiciones iniciales
x01 = np.array([0., 0., 0.1, 0.1])

# Definimos los parámetros del modelo
a = 1 # tasa de producción de mRNA [uM/h]
d = 0.1 # tasa de degradación de mRNA [1/h]
b = 2 # tasa de producción de proteína y por mRNA [uM/h]
c = 2 # tasa de producción de proteína medida por y [uM/h]
p = 0.05 # tasa de degradación de proteína y [1/h]
kon = 1 # constante de asociación [uM]
koff = 1 # constante de disociación [uM]
args = (a, d, b, p, K, m, kon, koff)

# Definimos el tiempo de simulación
t = np.linspace(0,40,200)

# Resolvemos
x1 = scipy.integrate.odeint(repressor_txtl, x01, t, args = args)
x1 = x1.transpose()

# Ploteamos
plots[0].line(t, x1[0,:], line_width = 3, color = 'black')
plots[0].title.text = 'Expresión de mRNA'
plots[0].yaxis.axis_label = 'mRNA [um/h]'

plots[1].line(t, x1[1,:], line_width = 3, color = 'orange')
plots[1].title.text = 'Expresión de y'
plots[1].yaxis.axis_label = 'y [um/h]'

# Visualizamos
bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
#@title Simulaciones estocásticas de TX-TL

# Stoichiometry matrix
simple_update = np.array(
    [
        [ 1, 0, 0, 0], # p_free -> mRNA
        [-1, 0, 0, 0], # mRNA -> 0
        [ 0, 1, 1,-1], # p -> p_free + y (forward)
        [ 0,-1,-1, 1], # p_free + y -> p (reverse)
        [ 0, 1, 0, 0], # mRNA -> y
        [ 0,-1, 0, 0], # y -> 0
    ]
)

# Propensities matrix
def simple_propensity(propensities, population, t, a, d, b, p, K, m, kon, koff, Omega):
  m, y, p_free, p = population

  propensities[0] = a * p_free
  propensities[1] = d * m
  propensities[2] = koff * p
  propensities[3] = kon * y * p_free
  propensities[4] = b * m
  propensities[5] = p * y

# Definimos el entorno de visualización en bokeh
plot = bokeh.plotting.figure(width = 325*2, height = 275,
                          x_range = Range1d(0, 20),
                          x_axis_label = 'Tiempo [h]',
                          y_axis_label = 'y [# moléculas]')

# Definimos los parámetros del modelo
a = 1 # tasa de producción de mRNA [uM/h]
d = 0.1 # tasa de degradación de mRNA [1/h]
b = 2 # tasa de producción de proteína y por mRNA [uM/h]
c = 2 # tasa de producción de proteína medida por y [uM/h]
p = 0.05 # tasa de degradación de proteína y [1/h]
kon = 1 # constante de asociación [uM]
koff = 1 # constante de disociación [uM]

# Definimos los parámetros del algoritmo de Gillespie
Omega = 600 # 600
time_points = np.linspace(0, 20, 101)
population_0 = np.array([0., 0., 0.1*Omega, 0.1*Omega]) # condiciones iniciales
args1 = (a, d, b, p, K, m, kon, koff, Omega)

# Resolvemos mediante algoritmo de Gillespie
samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=10, args=args1, n_threads=2,progress_bar=True)

# Ploteamos la dinámica de la proteína y
for i in range(4):
  plot.line(time_points, samples[i,:,1], line_width = 1, color = 'black', alpha = 0.2, line_join = 'bevel')

# Visualizamos
bokeh.io.show(plot)

100%|██████████| 10/10 [00:06<00:00,  1.52it/s]


Como tarea adicional, intentemos explorar estas preguntas:

* En el *toggle switch*, ¿qué ocurre si consideramos una mutua **activación** en vez de represión?
* En este mismo modelo, consideremos las especies $Y_1$ y $Y_2$. ¿Qué ocurre si $Y_1$ reprime la expresión de $Y_2$, pero $Y_2$ induce la expresión de $Y_1$?
* Finalmente, considerando el *phase plane* del *toggle switch*, ¿qué ocurre si disminuimos la cooperatividad? (**Hint**: probar para $m = 2$ y $m = 1$)

In [ ]:
#@title La mutua activación también genera un sistema biestable!

# Definimos una función que captura las EDOs
def mutual_induction(X, t, a, d, K, m):
  """
  Las expresiones al lado derecho muestran la producción y degradación.
  El output de la función será dy / dt
  """

  y1, y2 = X

  return (
      [
          a * y1**m / (K**m + y2**m) - d*y1, # d y1 / dt
          a * y2**m / (K**m + y1**m) - d*y2, # d y2 / dt
      ]
  )

# Definimos las ecuaciones de nullclines de cada especie
def null_y1(y2_2, a, d, K, m):

  y1_2 = (a/d) * y2_2**m / (K**m + y2_2**m)

  return y1_2

def null_y2(y1_1, a, d, K, m):

  y2_1 = (a/d) * y1_1**m / (K**m + y1_1**m)

  return y2_1

# Definimos el entorno de visualización en bokeh
p = bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 3.1),
                          y_range = Range1d(0, 3.1),
                          x_axis_label = 'y1',
                          y_axis_label = 'y2')

# Definimos las variables de estados
y1_1 = np.linspace(0,3,300)
y2_2 = np.linspace(0,3,300)

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 1 # constante de disociación [uM]
m = 3 # cooperatividad

# Calculamos las expresiones del nullclines
y1_2 = null_y1(y2_2, a, d, K, m)
y2_1 = null_y2(y1_1, a, d, K, m)

# Podemos hallar la intersección mediante la función intersect
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

# Ploteamos
p.line(y1_1, y2_1, line_width = 3, color = 'black', alpha = 0.75)
p.line(y1_2, y2_2, line_width = 3, color = 'black', alpha = 0.75)
p.circle(y1_i, y2_i, line_width = 2, size = 8, color = 'black', fill_color = 'white')

# Visualizamos
bokeh.io.show(p)

In [ ]:
#@title La mutua represión y activación resulta en oscilaciones

# Definimos una función que captura las EDOs
def dual_feedback(X, t, a, d, K, m):
  """
  Las expresiones al lado derecho muestran la producción y degradación.
  El output de la función será dy / dt
  """

  y1, y2 = X

  return (
      [
          a * K**m / (K**m + y2**m) - d*y1, # d y1 / dt
          a * y1**m / (K**m + y1**m) - d*y2, # d y2 / dt
      ]
  )

# Definimos el entorno de visualización en bokeh
p = bokeh.plotting.figure(width = 325*2, height = 275,
                          x_range = Range1d(0, 20),
                          x_axis_label = 'Tiempo [h]',
                          y_axis_label = 'y1 [uM]')

# Definimos las condiciones iniciales
x01 = np.array([0., 0.,])

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 1 # constante de disociación [uM]
m = 3 # cooperatividad
args = (a, d, K, m)

# Definimos el tiempo de simulación
t = np.linspace(0,20,200)

# Resolvemos
x1 = scipy.integrate.odeint(dual_feedback, x01, t, args = args)
x1 = x1.transpose()

# Ploteamos
p.line(t, x1[0,:], line_width = 3, color = 'orange', legend_label = 'y1')
p.line(t, x1[1,:], line_width = 3, color = 'grey', legend_label = 'y2')
p.title.text = 'Dinámica del sistema'
p.legend.location = 'bottom_right'

# Visualizamos
bokeh.io.show(p)

In [ ]:
#@title Disminuir la cooperatividad resulta en un sistema monoestable

# Definimos las ecuaciones de nullclines de cada especie
def null_y1(y2_2, a, d, K, m):

  y1_2 = (a/d) * K**m / (K**m + y2_2**m)

  return y1_2

def null_y2(y1_1, a, d, K, m):

  y2_1 = (a/d) * K**m / (K**m + y1_1**m)

  return y2_1

# Definimos el entorno de visualización en bokeh
p = bokeh.plotting.figure(width = 325, height = 275,
                          x_range = Range1d(0, 3.1),
                          y_range = Range1d(0, 3.1),
                          x_axis_label = 'y1',
                          y_axis_label = 'y2')

# Definimos las variables de estados
y1_1 = np.linspace(0,3,300)
y2_2 = np.linspace(0,3,300)

# Definimos los parámetros del modelo
a = 2.2 # tasa de producción [uM/h]
d = 1 # tasa de degradación [1/h]
K = 1 # constante de disociación [uM]
m = 1 # cooperatividad

# Calculamos las expresiones del nullclines
y1_2 = null_y1(y2_2, a, d, K, m)
y2_1 = null_y2(y1_1, a, d, K, m)

# Podemos hallar la intersección mediante la función intersect
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

# Ploteamos
p.line(y1_1, y2_1, line_width = 3, color = 'black', alpha = 0.75)
p.line(y1_2, y2_2, line_width = 3, color = 'black', alpha = 0.75)
p.circle(y1_i, y2_i, line_width = 2, size = 8, color = 'black', fill_color = 'white')

# Visualizamos
bokeh.io.show(p)